In [1]:
import numpy as np
import xsimlab as xs
#from scipy.integrate import odeint
import matplotlib.pyplot as plt

In [2]:
plt.rcParams['figure.figsize'] = [15, 10]

# OKAY!
so actually this higher order grid makes shit more complicated than it needs to be!
instead, keep track of component dims within component, keep state of components within components, BUT, with each added "box" or PhysEnv,
tha dimensionality of each Component changes, and stays coherent!

Fluxes can automatically be calcualted over all PhysEnvs/Boxes this way, vectorization..!



In [76]:
@xs.process
class HigherOrder:
    """so this keeps track of total dims, gives these to sub processes, which are components here"""
    dim = xs.variable(intent='in')
    H = xs.index(dims='H')
    
    def initialize(self):
        self.H = np.arange(self.dim)
    pass
    
@xs.process
class Comp:
    H_dim = xs.foreign(HigherOrder,'dim')
    
            
    @xs.runtime(args="step_delta")
    def run_step(self, dt):
        print(self.label,'step')
        self.delta = sum((v for v in self.fluxes)) * dt  # multiply by time step
        print('delta', self.delta)
    
    @xs.runtime(args="step_delta")
    def finalize_step(self, dt):
        print(self.label,'fin')
        self.state += self.delta * self.state
        print('state', self.state)

@xs.process
class Nutrient(Comp):
    """so this keeps track of one type of component, state as numpy array 'inout', udpated here
    collects all fluxes acting on component, and provides states to fluxes to compute it
    
    importantly the components reflect the dimensionality of the full model! this way vectorisation takes care of all else (i hope)
    """
    label= xs.variable(default='N')
    dim = xs.variable(intent='in')
    N = xs.index(dims='N')
    
    state = xs.variable(dims=('H','N'), intent='inout')
    
    fluxes = xs.group('N_flux')
    
    #flowrate = xs.variable(dims=[(),('N'),('H','N')], intent='in')
    
    def initialize(self):
        self.N = np.arange(self.dim)
        print('N', self.N, self.state)

@xs.process
class Phytoplankton(Comp):
    """so this keeps track of one type of component, state as numpy array 'inout', udpated here
    collects all fluxes acting on component, and provides states to fluxes to compute it
    
    importantly the components reflect the dimensionality of the full model! this way vectorisation takes care of all else (i hope)
    """
    label= xs.variable(default='P')
    dim = xs.variable(intent='in')
    P = xs.index(dims='P')
    
    state = xs.variable(dims=('H','P'), intent='inout')
    
    fluxes = xs.group('P_flux')
    
    halfsat = xs.variable(dims=[(),('P'),('H','P')], intent='in')
    
    def initialize(self):
        self.P = np.arange(self.dim)
        print('P', self.P, self.state)


@xs.process
class Zooplankton(Comp):
    """so this keeps track of one type of component, state as numpy array 'inout', udpated here
    collects all fluxes acting on component, and provides states to fluxes to compute it
    
    importantly the components reflect the dimensionality of the full model! this way vectorisation takes care of all else (i hope)
    """    
    label= xs.variable(default='Z')
    dim = xs.variable(intent='in')
    Z = xs.index(dims='Z')
    
    state= xs.variable(dims=('H','Z'), intent='inout')
    
    fluxes = xs.group('Z_flux')
    
    def initialize(self):
        self.Z = np.arange(self.dim)
        print('Z',self.Z,self.state)


@xs.process
class Flux:
    """This is a flux, that collects component states and gives flux back to the components
    """
    N_flux = xs.variable(dims= ('H','N'), intent='out', groups='N_flux')
    P_flux = xs.variable(dims= ('H','P'), intent='out', groups='P_flux')
    Z_flux = xs.variable(dims= ('H','Z'), intent='out', groups='Z_flux')
    
    P_halfsat = xs.foreign(Phytoplankton, 'halfsat')
    
    N = xs.foreign(Nutrient, 'state')
    P = xs.foreign(Phytoplankton, 'state')
    Z = xs.foreign(Zooplankton, 'state')
    
    #@xs.runtime(args='step_delta')
    def run_step(self):
        P_nutlim = self.N/(self.P_halfsat + self.N) 
        print('nutlim',P_nutlim, np.sum(P_nutlim * self.P, axis = 1, keepdims = True))
        Z_mortality = 0.1
        
        self.N_flux = - np.sum(P_nutlim * self.P, axis = 1, keepdims = True)
        self.P_flux = P_nutlim * self.P
        
        self.Z_flux = - Z_mortality * self.Z
    

@xs.process
class ModelSetup:
    P_state = xs.foreign(Phytoplankton, 'state', intent='out')
    Z_state = xs.foreign(Zooplankton, 'state', intent='out')
    N_state = xs.foreign(Nutrient, 'state', intent='out')
    
    P_dim = xs.foreign(Phytoplankton, 'dim', intent='out')
    Z_dim = xs.foreign(Zooplankton, 'dim', intent='out')
    N_dim = xs.foreign(Nutrient, 'dim', intent='out')
    
    H_dim = xs.foreign(HigherOrder,'dim')
    
    dimP = xs.variable(intent='in')
    dimZ = xs.variable(intent='in')
    dimN = xs.variable(intent='in') 
    
    initValP = xs.variable(intent='in')
    initValZ = xs.variable(intent='in')
    initValN = xs.variable(intent='in')
    
    P_halfsat = xs.foreign(Phytoplankton, 'halfsat', intent='out')
    
    def initialize(self):
        self.P_dim = self.dimP
        self.Z_dim = self.dimZ
        self.N_dim = self.dimN
        self.P_state = np.full((self.H_dim,self.P_dim), self.initValP, dtype='float64')
        self.Z_state = np.full((self.H_dim,self.Z_dim), self.initValZ, dtype='float64')
        self.N_state = np.linspace(0.0001, 1, self.H_dim*self.N_dim).reshape(self.H_dim,self.N_dim)
        
        self.P_halfsat = np.linspace(0,1,self.H_dim * self.P_dim).reshape(self.H_dim,self.P_dim)

In [77]:
Simple = xs.Model({'H':HigherOrder, 'N':Nutrient, 'P':Phytoplankton, 'Z':Zooplankton, 'Flx':Flux, 'MS':ModelSetup})
Simple

<xsimlab.Model (6 processes, 10 inputs)>
H
    dim          [in]
MS
    dimZ         [in]
    initValZ     [in]
    dimN         [in]
    initValP     [in]
    initValN     [in]
    dimP         [in]
Flx
N
    label        [in]
P
    label        [in]
Z
    label        [in]

In [78]:
Simple_in = xs.create_setup(
    model=Simple,
    clocks={
         'time': np.linspace(0., 2., 10)
    },
    master_clock='time',
    input_vars={
        # set full dims
        'H__dim':5,
        
        'MS__dimN':1,
        'MS__initValN':5,
        
        'MS__dimP':2,
        'MS__initValP':1,
        
        'MS__dimZ':3,
        'MS__initValZ':2
        
    },
    output_vars={
        'P__state':'time',
        'Z__state':'time'
    }
)

In [79]:
with Simple:
    sim_out = Simple_in.xsimlab.run()

N [0] [[1.00000e-04]
 [2.50075e-01]
 [5.00050e-01]
 [7.50025e-01]
 [1.00000e+00]]
P [0 1] [[1. 1.]
 [1. 1.]
 [1. 1.]
 [1. 1.]
 [1. 1.]]
Z [0 1 2] [[2. 2. 2.]
 [2. 2. 2.]
 [2. 2. 2.]
 [2. 2. 2.]
 [2. 2. 2.]]
nutlim [[1.00000000e+00 8.99190728e-04]
 [5.29486493e-01 4.28644889e-01]
 [5.29436677e-01 4.73709140e-01]
 [5.29420069e-01 4.90917421e-01]
 [5.29411765e-01 5.00000000e-01]] [[1.00089919]
 [0.95813138]
 [1.00314582]
 [1.02033749]
 [1.02941176]]
N step
delta [[-0.22242204]
 [-0.21291808]
 [-0.22292129]
 [-0.22674166]
 [-0.22875817]]
P step
delta [[2.22222222e-01 1.99820162e-04]
 [1.17663665e-01 9.52544197e-02]
 [1.17652595e-01 1.05268698e-01]
 [1.17648904e-01 1.09092760e-01]
 [1.17647059e-01 1.11111111e-01]]
Z step
delta [[-0.04444444 -0.04444444 -0.04444444]
 [-0.04444444 -0.04444444 -0.04444444]
 [-0.04444444 -0.04444444 -0.04444444]
 [-0.04444444 -0.04444444 -0.04444444]
 [-0.04444444 -0.04444444 -0.04444444]]
N fin
state [[7.77577958e-05]
 [1.96829510e-01]
 [3.88578208e-01]
 [5.79

In [43]:
sim_out

<xarray.Dataset>
Dimensions:        (H: 5, N: 1, P: 2, Z: 3, time: 10)
Coordinates:
  * H              (H) int64 0 1 2 3 4
  * N              (N) int64 0
  * P              (P) int64 0 1
  * Z              (Z) int64 0 1 2
  * time           (time) float64 0.0 0.2222 0.4444 0.6667 ... 1.556 1.778 2.0
Data variables:
    Flx__flowrate  float64 0.1
    H__dim         int64 5
    MS__dimN       int64 1
    MS__dimP       int64 2
    MS__dimZ       int64 3
    MS__initValN   int64 5
    MS__initValP   int64 1
    MS__initValZ   int64 2
    N__label       <U1 'N'
    P__label       <U1 'P'
    P__state       (time, H, P) float64 1.0 1.0 1.0 1.0 ... 0.3648 0.3321 0.3043
    Z__label       <U1 'Z'
    Z__state       (time, H, Z) float64 2.0 2.0 2.0 2.0 ... 1.413 1.413 1.413